# Генерация тренировочного и тестового датасетов

Импортируем нужные библиотеки

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import scipy as sp

Прочитаем предварительно обработанный датасет и удалим все строки, с пропусками

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Анализ соц сетей/кр/Task_1_prepprocessed.csv')
df = df.dropna()
df


,class,date,from,to,subject,body
0,0,4,info@global-change.com,michelle.lokay@enron.com,next wave energi trade,energi industri profession global chang associ...
1,0,1,info@pmaconference.com,michelle.lokay@enron.com,regist next txu capac auction,regist next txu energi capac auction new regis...
2,0,6,info@pmaconference.com,michelle.lokay@enron.com,merchant power monthli free sampl,merchant power monthli month s issu almost mw ...
3,0,3,bruno@firstconf.com,energynews@fc.ease.lsoft.com,eyeforenergi updat,welcom week s eyeforenergi updat refresh memor...
4,0,1,deanrogers@energyclasses.com,michelle.lokay@enron.com,deriv earli bird til march houston,deriv energi profession two full day april ear...
...,...,...,...,...,...,...
30687,1,3,jacob rzucidlo <lavoneaker@stalag13.com>,johnny wynott <varou@iit.demokritos.gr>,cpu pain m edicati n ship d r,arrghh west amnstv amlsmith basu petrom qureai...
30688,1,5,hal leake <annettgaskell@buglover.net>,renato mooney <sigletos@iit.demokritos.gr>,dn troubl f r ee,dn troubl f r ee angiospasma zekauskasa anarti...
30689,1,2,dr collins khumalo <khumalo_20@sunumail.sn>,khumalo_20@sunumail.sn,dr collin khumalo,dr collin khumalo attn mr presid dr collin khu...
30690,1,6,Customer Support <support@citibank.com>,Paliourg <paliourg@iit.demokritos.gr>,dear custom detail compromis,dear custom detail compromis dear custom recen...


Сравним результаты, полученные при использовании трех различных способов векторизации:
<ol>
    <li>Базовый (в качестве предиктора используется только <code>body</code>).
    <li>В качестве предикторов используем объединенные признаки <code>subject</code> и <code>body</code>, а также день недели, полученный из колонки <code>date</code>.
    <li>Аналогично 1., только с использованием n-грамм.
</ol>

Для базового алгоритма из загруженного датасета возьмем только колонку <code>body</code> и колонку <code>class</code> с обозначением класса сообщения (идет первой).

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 5], df.iloc[:,0], test_size=0.25, random_state=9)

Чтобы не модифицировать данные, использующиеся для базового алгоритма, сделаем глубокую копию данных для их дальнейшего преобразования. В частности, на копии произведем конкатенацию колонок `subject` и `body`. При разделении копии на тренировочный и тестовый наборы используем тот же `random_state`, что и для основого алгоритма, в целях сохранения соответствия полученных наборов.

In [ ]:
df_copy = df.copy(deep = True)

for i in range(len(df_copy)):
    if df.iat[i, 4] != '':
        try:
            df_copy.iat[i, 5] = df_copy.iat[i, 4] + ' ' + df_copy.iat[i, 5]
        except:
            print("Something got wrong!")

x_train_b, x_test_b, _, _ = train_test_split(
    df_copy.iloc[:, [1, 5]], df_copy.iloc[:,0], test_size=0.25, random_state=9)

# y_train_b и y_test_b совпадают с y_train, y_test. Хранить их отдельно надобности нет.

# Генерация признаков

Для базового алгоритма сначала из тренировочного набора данных составим словарь, используемый для векторизации и генерации признаков TF-IDF, во время преобразования тренировочного набора данных. Затем этот же словарь используем в функции трансформации тестового набора. Предлагаемый класс <code>TfidfVectorizer</code> используем с параметрами по умолчанию.

In [ ]:
vectorizer_a = TfidfVectorizer()

x_train_a = vectorizer_a.fit_transform(x_train)
x_test_a = vectorizer_a.transform(x_test)

Для модифицированного алгоритма, где используется день недели, векторизацию произведем на колонке конкатенированных темы письма и содержания. К результату выполнения векторизации добавим колонку индексов дней недели. В данном случае векторизатор также инициализируем с параметрами по умолчанию.

In [ ]:
vectorizer_b = TfidfVectorizer()
# Добавляем к полученной в результате векторизации разреженной матрице столбец со значениями дней недели
x_train_b = sp.sparse.hstack((vectorizer_b.fit_transform(x_train_b.iloc[:, 1]), x_train_b.iloc[:, 0].values.reshape(len(x_train_b.iloc[:, 0]),1)))
x_test_b = sp.sparse.hstack((vectorizer_b.transform(x_test_b.iloc[:, 1]), x_test_b.iloc[:, 0].values.reshape(len(x_test_b.iloc[:, 0]),1)))

В третьем сценарии, где вместо слов используются биграммы для генерации признаков TF-IDF, при инициализации векторизатора укажем параметр <code>ngram_range = (2, 2)</code>.

In [ ]:
vectorizer_c = TfidfVectorizer(ngram_range = (2, 2))

x_train_c = vectorizer_c.fit_transform(x_train)
x_test_c = vectorizer_c.transform(x_test)

# Классификация

Инициализируем три классификатора с одинаковыми параметрами.

In [ ]:
rfc_a = RandomForestClassifier(random_state=9, n_jobs=10, n_estimators=7)
rfc_b = RandomForestClassifier(random_state=9, n_jobs=10, n_estimators=7)
rfc_c = RandomForestClassifier(random_state=9, n_jobs=10, n_estimators=7)

rfc_a.fit(x_train_a, y_train)
rfc_b.fit(x_train_b, y_train)
rfc_c.fit(x_train_c, y_train)

RandomForestClassifier(n_estimators=7, n_jobs=10, random_state=9)

# Расчет метрик
Для получения метрик удобнее всего воспользоваться модулем `sklearn.metrics`. Выполним предсказания на тестовых данных и выполним оценку полученных моделей для трех случаев.

In [ ]:
y_pred_a = rfc_a.predict(x_test_a)
y_pred_b = rfc_b.predict(x_test_b)
y_pred_c = rfc_c.predict(x_test_c)


print(classification_report(y_test, y_pred_a, digits=3))
print(classification_report(y_test, y_pred_b, digits=3))
print(classification_report(y_test, y_pred_c, digits=3))

              precision    recall  f1-score   support

           0      0.960     0.977     0.968      3968
           1      0.973     0.954     0.963      3505

    accuracy                          0.966      7473
   macro avg      0.967     0.965     0.966      7473
weighted avg      0.966     0.966     0.966      7473

              precision    recall  f1-score   support

           0      0.961     0.981     0.971      3968
           1      0.978     0.954     0.966      3505

    accuracy                          0.969      7473
   macro avg      0.969     0.968     0.968      7473
weighted avg      0.969     0.969     0.969      7473

              precision    recall  f1-score   support

           0      0.978     0.954     0.965      3968
           1      0.949     0.975     0.962      3505

    accuracy                          0.964      7473
   macro avg      0.963     0.964     0.964      7473
weighted avg      0.964     0.964     0.964      7473



Для примера сравним первые два сценирия с точки зрения FPR и precision.
Для получения FPR достаточно сгенерировать confusion matrix и рассчитать его на основе значений из матрицы, в то время как precision может быть получен с помощью отдельной функции.

In [ ]:
tn_a, fp_a, _, _  = confusion_matrix(y_test, y_pred_a).ravel()
tn_b, fp_b, _, _ = confusion_matrix(y_test, y_pred_b).ravel()

fpr_a = fp_a / (fp_a + tn_a)
fpr_b = fp_b / (fp_b + tn_b)

pr_a = precision_score(y_test, y_pred_a)
pr_b = precision_score(y_test, y_pred_b)

print(f'Difference in FPR: {fpr_b - fpr_a}')
print(f'Difference in precision: {pr_b - pr_a}') функции.

[ ]
tn_a, fp_a, _, _  = confusion_matrix(y_test, y_pred_a).ravel()
tn_b, fp_b, _, _ = confusion_matrix(y_test, y_pred_b).ravel()

fpr_a = fp_a / (fp_a + tn_a)
fpr_b = fp_b / (fp_b + tn_b)

pr_a = precision_score(y_test, y_pred_a)
pr_b = precision_score(y_test, y_pred_b)

print(f'Difference in FPR: {fpr_b - fpr_a}')

Difference in FPR: -0.000423011844331642
Difference in precision: 0.0005386334269830151


Difference in FPR: -0.000423011844331642
Difference in precision: 0.0005386334269830151
